In [4]:
!pip install openai
!pip install langchain
!pip install "weaviate-client==3.*"
!pip install sentence-transformers
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 2.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 4.5 MB/s eta 0:00:0000:0100:01
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.4/924.4 kB 3.1 MB/s eta 0:00

In [45]:
import langchain

In [46]:
langchain.__version__

'0.0.336'

In [29]:
import openai
import weaviate
import pandas as pd
#from google.colab import drive
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Weaviate
from langchain.chains import RetrievalQA
from langchain.agents.types import AgentType
from langchain.agents import AgentExecutor, Tool,initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import AzureChatOpenAI
from langchain.retrievers import WeaviateHybridSearchRetriever

In [17]:
azure_openai_endpoint="https://pagmarketingopenai.openai.azure.com/"
#"https://pagmarketingdemov2.openai.azure.com/"
azure_openai="0c5adeb8121b4602b3e0735ba4a06ef9"
#"02f634fe573b4c52a0f74f2dead5c3b8"
azure_openai_engine="pagmarketinggpt"

In [18]:
llm = AzureChatOpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key = azure_openai,
    api_version = "2023-03-15-preview",
    azure_endpoint = azure_openai_endpoint,
    model=azure_openai_engine,
    temperature=0.7,
    max_tokens=800,
)

In [65]:
WEAVIATE_URL = "https://ragtestarray-4gihzxpr.weaviate.network"
WEAVIATE_API_KEY = "7E0Vf7POMdgUkpQfEHj5hPMpfUtPxNCNIisB"

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY),
)

In [53]:
temp_1 = []
for name in ["flu", "hpv"]:
    df = pd.read_csv('data/scrape_' + name + '.csv', index_col=0)
    temp_1.append(df)

In [54]:
data_new = pd.concat(temp_1)

In [55]:
data_new.head(2)

,id,type,author,subreddit,created_at,content
0,k8lq6b4,comment,sp00kens_,texts,1699589290,Sorry dude. She literally said she didn’t see ...
1,k8ks91w,comment,sp00kens_,deadbydaylight,1699574872,Yea I think the ranking system is weird. I don...


In [13]:
temp = []
for name in ["data/kidney_cancer_stories_v2.txt", "data/cd_stories_v2.txt", "data/hcl_stories_v2.txt", "data/hpv_stories_v2.txt"]:
    data = pd.read_csv(name, index_col=0)
    temp.append(data)

In [14]:
data = pd.concat(temp)

In [15]:
data.head()

,Story,Name
0,Sarah was diagnosed with kidney cancer three m...,The patient's name is Sarah.
1,"Meet Sarah, a 48-year-old kidney cancer patien...",The patient name in the story is Sarah.
2,Meet Sarah. She's a 53-year-old woman who was ...,The patient's name is Sarah.
3,Emily is a kidney cancer patient who was diagn...,The patient's name is Emily.
4,Samantha’s journey with kidney cancer began wi...,The patient's name is Samantha.


In [24]:
data['Name_clean'] = [name.replace("The patient's name is ", "").replace("The patient name in the story is","").replace("Patient name: ","").replace("The patient's name in the story is","").replace("The patient's name in this story is ","").replace("Patient Name: ","").replace("The patient name is ","").strip() for name in data['Name']]

In [25]:
data.head()

,Story,Name,Name_clean
0,Sarah was diagnosed with kidney cancer three m...,The patient's name is Sarah.,Sarah.
1,"Meet Sarah, a 48-year-old kidney cancer patien...",The patient name in the story is Sarah.,Sarah.
2,Meet Sarah. She's a 53-year-old woman who was ...,The patient's name is Sarah.,Sarah.
3,Emily is a kidney cancer patient who was diagn...,The patient's name is Emily.,Emily.
4,Samantha’s journey with kidney cancer began wi...,The patient's name is Samantha.,Samantha.


In [7]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {"device": "cuda"} mps
#model_kwargs = {"device": "mps"}
model_kwargs = {}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/huggingface_hub/inference/_text_generation.py:121: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator("best_of")
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/huggingface_hub/inference/_text_generation.py:140: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @validator("repetiti

In [56]:
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)

In [57]:
all_docs_reddit = []
# vectors_docs = []
count = 0
for index, d in data_new.iterrows():
    try:
        #list_docs = text_splitter.create_documents([d["text"]])
        #split = text_splitter.split_documents(list_docs)
        try:
            # base_docs = text_splitter.split_text(d['headline'] + d["text"])
            # list_docs = [d['title'] + '##' + base_docs[i] for i in range(0, len(base_docs))]
            base_docs = text_splitter.split_text(d['content'])
            list_docs = [d['author'] + '##' + d['subreddit'] + '##' + base_docs[i] for i in range(0, len(base_docs))]
            create_docs = text_splitter.create_documents(list_docs)
            #split_docs = text_splitter.split_documents(list_docs)

        except Exception as e:
            #print(e)
            list_docs = []
            create_docs = []
            split_docs = []
            continue

        # try:
        #     vector_list = embeddings.embed_documents(list_docs)
        # except:
        #     vector_list = []
        all_docs_reddit.extend(create_docs)
        # vectors_docs.append(vector_list)
        count = count + 1
    except Exception as  e:
        print(e)
        continue

In [58]:
len(all_docs_reddit)

80434

In [26]:
all_docs = []
# vectors_docs = []
count = 0
for index, d in data.iterrows():
    try:
        #list_docs = text_splitter.create_documents([d["text"]])
        #split = text_splitter.split_documents(list_docs)
        try:
            # base_docs = text_splitter.split_text(d['headline'] + d["text"])
            # list_docs = [d['title'] + '##' + base_docs[i] for i in range(0, len(base_docs))]
            base_docs = text_splitter.split_text(d['Story'])
            list_docs = [d['Name_clean'] + '##' + base_docs[i] for i in range(0, len(base_docs))]
            create_docs = text_splitter.create_documents(list_docs)
            #split_docs = text_splitter.split_documents(list_docs)

        except Exception as e:
            #print(e)
            list_docs = []
            create_docs = []
            split_docs = []
            continue

        # try:
        #     vector_list = embeddings.embed_documents(list_docs)
        # except:
        #     vector_list = []
        all_docs.extend(create_docs)
        # vectors_docs.append(vector_list)
        count = count + 1
    except Exception as  e:
        print(e)
        continue

In [27]:
all_docs[0]

Document(page_content='Sarah.##Sarah was diagnosed with kidney cancer three months ago, after experiencing severe abdominal pain. She had to go through several tests and procedures, but finally got a diagnosis. The news was devastating for Sarah and her family. \n\nThe doctor recommended surgery to remove the tumor, which was scheduled a few weeks later. Sarah underwent the procedure successfully, and she felt relieved that the cancer had been removed. However, she experienced some side effects, such as pain, fatigue, and nausea. The doctor prescribed some pain medication and advised her to rest and eat a healthy diet. \n\nAfter a few days, Sarah started feeling better')

In [32]:
for i in client.schema.get()['classes']:
    print(i['class'])
    #client.schema.delete_class(i['class'])

Patient_stories


In [8]:
# patient_schema =   {
#       "class": "Patient_stories",
#       "description": "A class called document",
#       "vectorizer": embeddings,
#       }

In [28]:
vector_db_ps = Weaviate.from_documents(
    all_docs, client=client, by_text=False,index_name="Patient_stories", embedding=embeddings
)

In [59]:
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 


In [60]:
# How many elements each 
# list should have 
n = 1000
  
x = list(divide_chunks(all_docs_reddit, n)) 

In [62]:
len(x)


81

In [63]:
count = 0

In [66]:
for docs in x:
    vector_db_reddit = Weaviate.from_documents(
        docs, client=client, by_text=False,index_name="Reddit_stories", embedding=embeddings
    )
    count = count + len(docs)

    if count%2000 == 0:
        print("Done " + str(count))

Done 2000
Done 4000
Done 6000
Done 8000
Done 10000
Done 12000
Done 14000
Done 16000
Done 18000
Done 20000
Done 22000
Done 24000
Done 26000
Done 28000
Done 30000
Done 32000
Done 34000
Done 36000
Done 38000
Done 40000
Done 42000
Done 44000
Done 46000
Done 48000
Done 50000
Done 52000
Done 54000


[ERROR] Batch ConnectionError Exception occurred! Retrying in 2s. [1/3]


Done 56000
Done 58000
Done 60000
Done 62000
Done 64000
Done 66000
Done 68000
Done 70000
Done 72000
Done 74000
Done 76000
Done 78000
Done 80000


In [51]:
weaviate_obj = Weaviate(client, "Patient_stories", text_key="text")

In [38]:
weaviate_obj = Weaviate(client, "Patient_stories", text_key="text", embedding=embeddings)

In [52]:
weaviate_obj.similarity_search("HPV")

ValueError: _embedding cannot be None for similarity_search_with_score

In [35]:
# vector_retriever = WeaviateHybridSearchRetriever(
#     client=client,
#     index_name="Patient_stories",
#     text_key="text",
#     attributes=[],
#     create_schema_if_missing=True,
#     k=4
    
# )

In [ ]:
vector_retriever

In [209]:
#weaviate_obj.similarity_search("HPV")
#vector_db.as_retriever(search_kwargs={"k": 4})

In [67]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db_reddit.as_retriever(search_kwargs={"k": 4}), return_source_documents=True, verbose=True
)
#vector_db.as_retriever(search_kwargs={"k": 4})

In [68]:
response = qa_chain("What is flu?")



> Entering new RetrievalQA chain...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

> Finished chain.


/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


In [69]:
response

{'query': 'What is flu?',
 'result': 'The flu, or influenza, is a contagious respiratory illness caused by influenza viruses. It can cause mild to severe illness and can be particularly dangerous for certain populations, such as young children, older adults, and individuals with underlying health conditions. Symptoms can include fever, cough, sore throat, muscle aches, and fatigue.',
 'source_documents': [Document(page_content='Particular-Ad3934##flu##Whats flu B'),
  Document(page_content='CantoErgoSum##HPV##Covid IS NOT "just a flu." It\'s a CORONAVIRUS which most respiratory diseases are. It\'s a severe and novel and ever-changing virus with no cure and very little mitigation in place.'),
  Document(page_content='anthropocene-##flu##I’m laid up in bed and my body is so fatigued with this flu. I thought I would just track how I’m feeling here each day.\n\nI don’t know when I started having the flu but it could’ve been any day last week maybe Monday. Which means I’ve had it for about 

In [238]:
dict(response['source_documents'][0])['page_content'].split("##")

['Sarah.',
 "Hi, my name is Sarah and I was recently diagnosed with Hairy Cell Leukemia. It's been a tough journey, but I'm grateful for the support of my family and medical team.\n\nThe first sign that something was wrong was when I noticed that I was always tired. At first, I thought it was because of work, but even on my days off, I felt exhausted. I went to the doctor and after a series of tests, I was diagnosed with Hairy Cell Leukemia.\n\nMy medical team immediately started me on treatment with cladribine, a chemotherapy drug that specifically targets the abnormal"]

In [239]:
name_value, content = dict(response['source_documents'][0])['page_content'].split("##")

In [ ]:
df[df['A'].str.contains("Hello|Britain")==True]

In [244]:
data[(data.Name_clean == name_value) & (data['Story'].str.contains(content)==True)]

,Story,Name,Name_clean
48,"Hi, my name is Sarah and I was recently diagno...",The patient's name is Sarah.,Sarah.


In [253]:
rows = []
for source in response['source_documents']:
    name_value, content = dict(source)['page_content'].split("##")
    df = data[(data.Name_clean == name_value) & (data['Story'].str.contains(content)==True)].copy()
    df['excerpt'] = content
    rows.append(df)

row_df = pd.concat(rows)
    

In [254]:
row_df.drop_duplicates(inplace=True)

In [256]:
row_df.iloc[0]['

Story         Hi, my name is Sarah and I was recently diagno...
Name                               The patient's name is Sarah.
Name_clean                                               Sarah.
excerpt       Hi, my name is Sarah and I was recently diagno...
Name: 48, dtype: object

In [63]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key='input', return_messages=True, output_key='output')

In [64]:
system_message = """
"You are the XYZ bot."
"This is conversation with a human. Answer the questions you get based on the knowledge you have."
"If you don't know the answer, just say that you don't, don't try to make up an answer."
"""

In [65]:
tools = [
        Tool(
            name="doc_search_tool",
            func=qa_chain,
            description=(
               "This tool is used to retrieve information from the knowledge base"
            )
        )
    ]

# agent = initialize_agent(
#         agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
#         tools=tools,
#         llm=llm,
#         memory=memory,
#         return_source_documents=True,
#         return_intermediate_steps=True,
#         agent_kwargs={"system_message": system_message}
#         )
agent = initialize_agent(
        agent = AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        tools=tools,
        llm=llm,
        memory=memory,
        return_source_documents=True,
        return_intermediate_steps=True,
        agent_kwargs={"system_message": system_message}
        )

In [66]:
result1 = agent("What are side effects of hairy cell leukemia?")

/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)




> Entering new RetrievalQA chain...


/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)



> Finished chain.


/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


In [39]:
agent("Can you refine to single list")

/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)




> Entering new RetrievalQA chain...


/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)



> Finished chain.


/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/langchain/chat_models/azure_openai.py:246: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/Users/kanchanapadmanabhan/Documents/dev/pagmarketing/rag_app_v2/venv/lib/python3.11/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


{'input': 'Can you refine to single list',
 'chat_history': [HumanMessage(content='What are side effects of kidney cancer?'),
  AIMessage(content='The side effects of kidney cancer can include lower back pain, blood in the urine, fatigue, weight loss, and swelling in the ankles or legs. Treatment options, such as surgery, chemotherapy, targeted therapy, or radiation therapy, can also cause side effects such as nausea, vomiting, hair loss, and fatigue. However, it is important to note that not all patients may experience all of these side effects and their severity can vary from person to person.'),
  HumanMessage(content='Can you refine to single list'),
  AIMessage(content="The common symptoms of kidney cancer include:\n- Blood in urine\n- Lower back pain or pain on one side of the body\n- Unintentional weight loss\n- Fatigue\n- Loss of appetite\n- Persistent fever\n\nThe side effects of kidney cancer treatment, such as chemotherapy, radiation therapy, and surgery, can vary depending 